In [ ]:
import numpy as np
import random
from math import floor

In [ ]:

MIN_SIZE = 2
MAX_SIZE = 10



In [ ]:
#Function to assert the builded grid is solvable
def is_solvable(grid):
    flat_grid = grid.flatten() 
    
    inversions = 0
    for i in range(len(flat_grid)):
        for j in range(i + 1, len(flat_grid)):
            if flat_grid[j] and flat_grid[i] and flat_grid[i] > flat_grid[j]:
                inversions += 1
    return inversions % 2 == 0

def build_line(start_symb, stop_symb, sep_symb, line):
    return '\n%s%s%s\n' % (start_symb, sep_symb.join(line), stop_symb)

In [ ]:
# State : Correspond a la grid
# Actions: Dé deplacer :
#0: dé en haut
#1: dé a gauche
#2: dé en bas
#3: dé à droite

class Grid :
    def __init__(self,size=3) -> None:
        assert size>=MIN_SIZE and size <= MAX_SIZE
        self.size=size

        while True:
            grid=list(range(size**2))
            random.shuffle(grid)
            grid=np.asarray(grid).reshape(size, size)
            if is_solvable(grid):break
        self.state=grid


    def take_action(self,action):
      i, j = np.where(self.state == 0)
      if len(i) > 0 and len(j) > 0:
        i = i[0]
        j = j[0]
        if action == 0:
            if i > 0:
                self.state[i][j], self.state[i-1][j] = self.state[i-1][j], self.state[i][j]
        elif action == 2:
            if i < self.size-1:
                self.state[i][j], self.state[i+1][j] = self.state[i+1][j], self.state[i][j]
        elif action ==1:
            if j > 0:
                self.state[i][j], self.state[i][j-1] = self.state[i][j-1], self.state[i][j]
        elif action == 3:
            if j < self.size-1:
                self.state[i][j], self.state[i][j+1] = self.state[i][j+1], self.state[i][j]

    def is_finish(self):
        ended_grid=[i for i in range(1,self.size**2)]
        ended_grid.append(0)
        return np.array_equal(
            self.state,
            np.asarray(ended_grid).reshape(self.size,self.size)
            )
    
    def get_possible_actions(self)-> list[int]:
        """return the possible actions"""
        actions=[i for i in range(0,4)]
        pos_empty=self.get_empty_position()
        if pos_empty[0]==0 : actions.remove(0)
        elif pos_empty[0]==(self.size-1):actions.remove(2)
        if pos_empty[1]==0:actions.remove(1)
        elif pos_empty[1]==(self.size-1):actions.remove(3)
        return  actions
    
    def get_empty_position(self)->tuple:
        """Return the position od the tuple"""
        return floor(np.argmin(self.state)/self.size),np.argmin(self.state)%self.size

    def get_good_place(self):
      ended_grid=[i for i in range(1,self.size**2)]
      ended_grid.append(0)
      return (np.asarray(ended_grid).reshape(self.size,self.size) == self.state ).sum()



    def __str__(self) -> str:
        """Renderer"""
        tile_line = np.full(self.size, '─' * 4).tolist()
        horizontal_line = build_line('├', '┤', '┼', tile_line)
        first_horizontal_line = build_line('┌', '┐', '┬', tile_line)
        last_horizontal_line = build_line('└', '┘', '┴', tile_line)
        grid_to_show = first_horizontal_line
        for count, row in enumerate(self.state):
            grid_to_show += '│'
            for tile in row:
                if tile == 0:
                    tile = '  '
                grid_to_show += ' %s │' % '{0:>2}'.format(tile)
            if not count == self.size - 1:
                grid_to_show += horizontal_line
        grid_to_show += last_horizontal_line
        grid_to_show+=f"\n Possible actions: {self.get_possible_actions()}"
        return grid_to_show
        
    

In [ ]:
grid=Grid(size=3)
print(grid.state)
grid.is_finish()

[[0 7 6]
 [4 3 2]
 [5 1 8]]


False

In [ ]:
import numpy as np

class Qlearning:
    def __init__(self, actions, alpha=0.1, gamma=0.9, epsilon=0.1,size=2):
        self.Q = {}
        self.actions = actions
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon
        self.size=size**2

    def get_Q(self, state, action):
        if tuple(state.reshape(self.size)) not in self.Q:
            self.Q[tuple(state.reshape(self.size))] = np.zeros(len(self.actions))
        return self.Q[tuple(state.reshape(self.size))][action]

    def choose_action(self, state,possible_actions):
        if np.random.uniform() < self.epsilon:
            # Choose a random action
            
            return np.random.choice(possible_actions)
        else:
            # Choose the action with the highest Q-value
            q_values = [self.get_Q(state, a) for a in possible_actions]
            max_q = max(q_values)
            if q_values.count(max_q) > 1:
                # Multiple actions have the same highest Q-value so we choose one randomly
                
                return possible_actions[np.random.choice(np.flatnonzero(np.array(q_values) == np.array(q_values).max()))]
            else:
                return possible_actions[np.argmax(q_values)]

    def update(self, state, action, reward, next_state):
        old_Q = self.get_Q(state, action)
        next_max_Q = max([self.get_Q(next_state, a) for a in self.actions])
        new_Q = old_Q + self.alpha * (reward + self.gamma * next_max_Q - old_Q)
        #print(action,reward)
        self.Q[tuple(state.reshape(self.size))][action] = new_Q
        #print(self.Q)


In [ ]:
class Game:
    def __init__(self,size=2) -> None:
        self.grid=Grid(size)

        self.end=False
        self.round=0
        self.reward=0
        self.sum_reward=0
        self.actions=[]# list of all actions done in the game
        self.last_good=0
        self.number_good_place=0
        self.ligne=0
        self.col=0
        ended_grid=[i for i in range(1,self.grid.size**2)]
        ended_grid.append(0)
        self.ended_grid = np.asarray(ended_grid).reshape(self.grid.size,self.grid.size)

    def play_round(self,action):
        """function_agent : fonction de choix de l'agent"""
        reward=-1
        last_ligne=self.ligne
        last_col=self.col

        self.grid.take_action(action)

        if self.grid.is_finish():
          self.end = True
          reward += 50000
        
        if (self.grid.state[0] == self.ended_grid[0]).all():
          if self.ligne < 1:
            reward +=30
          self.ligne=1
          if (self.grid.state[1] == self.ended_grid[1]).all():
            if self.ligne < 2:
              reward+=60
            self.ligne=2
        else:
          self.ligne=0
        if self.ligne < last_ligne:
          reward -= 2500 # So thaht he cant delete a good row



        if (self.grid.state[:,:1] == self.ended_grid[:,:1]).all():
          if self.col < 1:
            reward +=30
          self.col=1
          if (self.grid.state[:,1:2] == self.ended_grid[:,1:2]).all():
            if self.col < 2:
              reward+=60
            self.col=2
        else:
          self.col=0
        if self.col < last_col:
          reward -= 2500 # so that he cant delete a good column
        


        if self.grid.get_good_place() == self.number_good_place:
          reward -= self.last_good # more we dont increase the number of well positioned stuff more reward we loose
          self.last_good += 1
        elif self.grid.get_good_place() < self.number_good_place:
          reward -= 10
       
        reward+=self.grid.get_good_place()
        self.number_good_place = self.grid.get_good_place()
        return reward,self.grid.state,action


    def play_game(self,qlearner,stop=100000,update=True):
        while not self.end:
            state=self.grid.state.copy()
            action = qlearner.choose_action(state,self.grid.get_possible_actions())

            reward,next_state,action=self.play_round(action)
            #print(state)
            if update: # Falsefor playing without training
              qlearner.update(state, action, reward, next_state)
            qlearner.update(state, action, reward, next_state)
            if self.round > stop:
              return False
              break
            self.round+=1
            #print(self.round)
        return True

    def __str__(self):
        game_to_show=f"---- {self.round} ----------\n"
        game_to_show+=self.grid.__str__()
        game_to_show+=f"\nReward Round{self.reward} \nReward cum:{self.sum_reward}"
        return game_to_show




In [ ]:
size=3
qlearner = Qlearning(actions=[0, 1, 2, 3],size=size)

In [ ]:
test=np.load("qlearner.npy",allow_pickle=True)
qlearner.Q=test.item()

In [ ]:
np.save("qlearner.npy",qlearner.Q)

In [ ]:
test=np.load("qlearner.npy",allow_pickle=True)
test2=test.item()

### 1000 games With max of 15000 round (epsilon 0.5)

In [ ]:
qlearner.epsilon=0.5

nb_game = 1000
win=0
for i in range(nb_game):
  game = Game(size=size)
  round=0
  res=game.play_game(qlearner,15000)
  if res:
    win+=1
  print(i,res)

0 False
1 True
2 False
3 False
4 False
5 False
6 True
7 False
8 False
9 True
10 False
11 True
12 True
13 False
14 True
15 False
16 False
17 False
18 False
19 True
20 False
21 False
22 True
23 True
24 False
25 True
26 False
27 True
28 False
29 True
30 False
31 False
32 False
33 False
34 False
35 False
36 False
37 False
38 False
39 False
40 True
41 True
42 True
43 False
44 True
45 False
46 True
47 False
48 False
49 False
50 True
51 False
52 False
53 False
54 False
55 False
56 True
57 False
58 False
59 False
60 False
61 False
62 False
63 True
64 False
65 False
66 False
67 False
68 True
69 True
70 False
71 False
72 False
73 False
74 False
75 False
76 True
77 True
78 False
79 False
80 True
81 True
82 False
83 True
84 False
85 True
86 False
87 True
88 True
89 True
90 False
91 True
92 True
93 True
94 False
95 False
96 False
97 True
98 True
99 True
100 False
101 True
102 False
103 True
104 False
105 False
106 False
107 False
108 True
109 False
110 False
111 False
112 False
113 True
114 False
1

In [ ]:
win

753

753 win out of 1000 (75% of grid solved in less than 15000 round with 50% of exploration)

In [ ]:
qlearner.epsilon=0.3

nb_game = 100
win=0
for i in range(nb_game):
  game = Game(size=size)
  round=0
  res=game.play_game(qlearner,1000)
  if res:
    win+=1
  print(i,res)

0 True
1 False
2 True
3 False
4 False
5 False
6 True
7 False
8 False
9 False
10 False
11 True
12 False
13 False
14 False
15 False
16 False
17 False
18 False
19 False
20 False
21 False
22 False
23 True
24 False
25 False
26 True
27 False
28 False
29 False
30 False
31 False
32 True
33 False
34 False
35 False
36 False
37 False
38 False
39 False
40 False
41 True
42 False
43 False
44 True
45 False
46 False
47 False
48 False
49 False
50 False
51 False
52 False
53 False
54 False
55 False
56 False
57 False
58 True
59 False
60 False
61 False
62 False
63 False
64 False
65 True
66 False
67 False
68 False
69 False
70 False
71 False
72 False
73 False
74 False
75 False
76 False
77 False
78 False
79 False
80 False
81 False
82 False
83 False
84 False
85 False
86 False
87 False
88 False
89 True
90 False
91 False
92 False
93 False
94 False
95 False
96 False
97 False
98 True
99 False


In [ ]:
win

11

### 1000 games With max of 15000 round (epsilon 0.4)

In [ ]:
qlearner.epsilon=0.4

nb_game = 1000
win=0
for i in range(nb_game):
  game = Game(size=size)
  round=0
  res=game.play_game(qlearner,500)
  if res:
    win+=1
  print(i,res)

0 False
1 False
2 False
3 False
4 False
5 False
6 False
7 False
8 False
9 False
10 False
11 False
12 False
13 False
14 False
15 False
16 False
17 False
18 False
19 False
20 False
21 True
22 False
23 True
24 False
25 False
26 False
27 False
28 False
29 False
30 False
31 False
32 False
33 False
34 False
35 False
36 False
37 False
38 False
39 False
40 False
41 False
42 False
43 False
44 False
45 False
46 False
47 False
48 False
49 False
50 False
51 False
52 False
53 False
54 False
55 False
56 True
57 False
58 False
59 False
60 False
61 False
62 False
63 False
64 False
65 False
66 False
67 False
68 False
69 False
70 False
71 True
72 False
73 False
74 False
75 False
76 False
77 False
78 False
79 False
80 True
81 True
82 False
83 False
84 False
85 False
86 False
87 False
88 False
89 False
90 False
91 False
92 True
93 False
94 False
95 False
96 False
97 False
98 False
99 False
100 False
101 False
102 False
103 False
104 False
105 False
106 False
107 True
108 False
109 False
110 False
111 Fals

In [ ]:
win

258

In [ ]:
qlearner.epsilon=0.1
game = Game(size=size)
game.grid.state=np.array([[0, 2, 3],
       [4, 1, 5],
       [7, 8, 6]])  
res=game.play_game(qlearner,10000,False)
print(game.round)

10001


In [ ]:
game.grid.state=np.array([[1, 2, 3],
       [4, 5, 5],
       [7, 8, 6]])  

In [ ]:
ended_grid=[i for i in range(1,3**2)]
ended_grid.append(0)
ended_grid = np.asarray(ended_grid).reshape(3,3)
      

In [ ]:
ended_grid[:,1:2]

array([[2],
       [5],
       [8]])

In [ ]:
(game.grid.state[:,1:2] == ended_grid[:,1:2]).all()

True

In [ ]:
qlearner.epsilon=0.9

nb_game = 100
win=0
for i in range(nb_game):
  game = Game(size=size)
  res=game.play_game(qlearner,25000)
  if res:
    win+=1
  print(i,res)

0 False
1 False
2 False
3 False
4 False


KeyboardInterrupt: ignored